# Analysis of tweet text

In [1]:
import pymongo
import re
import pprint
import logging

In [2]:
# Logging functionality
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logging.debug('Start of program')

2018-05-09 23:27:00,310 - DEBUG - Start of program


In [3]:
# Get the tweets stored in MongoDB
client = pymongo.MongoClient()
tweets = client['twitter']['tweets'].find()
latest_tweet = tweets[26]

In [4]:
def remove_from_word_list(original_list, list_subset):
    for each in list_subset:
        idx = original_list.index(each)
        del original_list[idx]
    return original_list

In [5]:
def print_important_content(tweet):
    print('By {} on {}\n'.format(tweet['user']['name'], tweet['created_at']))
    print('"{}"\n'.format(tweet['full_text']))
    print('{}\n'.format(tweet['entities']['urls'][0]['expanded_url']))
    pprint.pprint(tweet['entities'])
    
print_important_content(latest_tweet)

By The Guardian on Mon Apr 30 18:45:05 +0000 2018

"Labour suspends John Woodcock over sexual harassment claim https://t.co/7kpIXGuYTV"

https://trib.al/1TTGn0U

{'hashtags': [],
 'symbols': [],
 'urls': [{'display_url': 'trib.al/1TTGn0U',
           'expanded_url': 'https://trib.al/1TTGn0U',
           'indices': [59, 82],
           'url': 'https://t.co/7kpIXGuYTV'}],
 'user_mentions': []}


In [6]:
full_url = latest_tweet['entities']['urls'][0]['expanded_url']
full_url

'https://trib.al/1TTGn0U'

In [7]:
def hashtag(tweet):
    hashtag_list = []
    for tag in tweet['hashtags']:
        hashtag_list.append('#' + tag['text'])
    return hashtag_list

hashtags = hashtag(latest_tweet['entities'])
hashtags

[]

In [8]:
latest_tweet

{'_id': ObjectId('5ae766611c398f0d01baa6ee'),
 'created_at': 'Mon Apr 30 18:45:05 +0000 2018',
 'id': 991025665351868423,
 'id_str': '991025665351868423',
 'full_text': 'Labour suspends John Woodcock over sexual harassment claim https://t.co/7kpIXGuYTV',
 'truncated': False,
 'display_text_range': [0, 82],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/7kpIXGuYTV',
    'expanded_url': 'https://trib.al/1TTGn0U',
    'display_url': 'trib.al/1TTGn0U',
    'indices': [59, 82]}]},
 'source': '<a href="http://www.socialflow.com" rel="nofollow">SocialFlow</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 87818409,
  'id_str': '87818409',
  'name': 'The Guardian',
  'screen_name': 'guardian',
  'location': 'London',
  'description': 'The need for independent journalism has never been greater. Become a G

In [9]:
# Break the text content apart for better formatting.
words = latest_tweet['full_text'].split(' ')

In [10]:
# Display the broken down list of each 'word'.
words

['Labour',
 'suspends',
 'John',
 'Woodcock',
 'over',
 'sexual',
 'harassment',
 'claim',
 'https://t.co/7kpIXGuYTV']

In [11]:
words = remove_from_word_list(words, hashtags)
words

['Labour',
 'suspends',
 'John',
 'Woodcock',
 'over',
 'sexual',
 'harassment',
 'claim',
 'https://t.co/7kpIXGuYTV']

In [12]:
# Find any urls in the list of words.
# Located better regex for url pattern matching - http://www.noah.org/wiki/RegEx_Python
def url_locator(word_list):
    urls = []
    for word in word_list:
        url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', word)
        if url:
            urls.append(url[0])
    return urls
urls = url_locator(words)
urls

['https://t.co/7kpIXGuYTV']

In [13]:
words = remove_from_word_list(words, urls)
words

['Labour',
 'suspends',
 'John',
 'Woodcock',
 'over',
 'sexual',
 'harassment',
 'claim']

In [14]:
# remove @mentions
def detect_user_handles(word_list):
    user_handle = []
    for word in word_list:
        if word.startswith('@'):
            user_handle.append(word)
    return user_handle

users = detect_user_handles(words)
users

[]

In [15]:
words = remove_from_word_list(words, users)
words

['Labour',
 'suspends',
 'John',
 'Woodcock',
 'over',
 'sexual',
 'harassment',
 'claim']

In [16]:
# def remove_extra_characters(word_list):
def strip_nonalphanumeric_characters(word_list):
    clean_list = []
    regex = re.compile('[^a-zA-Z0-9]')
    for word in word_list:
        word = regex.sub('', word)
        if word != '':
            clean_list.append(word)
    return clean_list
clean_words = strip_nonalphanumeric_characters(words)
clean_words

['Labour',
 'suspends',
 'John',
 'Woodcock',
 'over',
 'sexual',
 'harassment',
 'claim']

In [17]:
list(set(words) - set(clean_words))

[]

In [18]:
logging.debug('End of program')

2018-05-09 23:27:00,442 - DEBUG - End of program
